In [ ]:
import polars as pl
import squarify
import rtsvg
rt = rtsvg.RACETrack()
edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.rt_layout.parquet'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df            = pl.DataFrame(_lu_)
df            = df.with_columns(pl.col('fm').cast(pl.String), pl.col('to').cast(pl.String))
g             = rt.createNetworkXGraph(df, [('fm','to')])
_nodes_       = list(set(df['fm']) | set(df['to']))
_node_colors_ = {}
for i in range(98): _node_colors_[_nodes_.pop(0)] = 'red'
for i in range(24): _node_colors_[_nodes_.pop(0)] = 'yellow'
for i in range(41): _node_colors_[_nodes_.pop(0)] = 'green'
for i in range(2):  _node_colors_[_nodes_.pop(0)] = 'black'

_pos1_  = rt.treeMapNodeColorPlacement(g, set(df['fm']) | set(df['to']), _node_colors_)
_pos2_  = rt.treeMapNodeColorPlacement(g, set(df['fm']) | set(df['to']), _node_colors_, collapse=True)

sz = {'df':df, 'w':256, 'h':256, 'relationships':[('fm','to')], 'link_shape':'curve'}
_tiles_ = []
_tiles_.append(rt.link    (pos=_pos1_, node_color=_node_colors_, **sz))
_tiles_.append(rt.link    (pos=_pos2_, node_color=_node_colors_, **sz))
_tiles_.append(rt.link    (pos=_pos2_,                           **sz))
_tiles_.append(rt.link    (pos=_pos2_, link_size='vary',         **sz))
_tiles_.append(rt.linkNode(pos=_pos1_, node_color=_node_colors_, **sz))
_tiles_.append(rt.linkNode(pos=_pos2_, node_color=_node_colors_, **sz))
_tiles_.append(rt.linkNode(pos=_pos2_,                           **sz))
_tiles_.append(rt.linkNode(pos=_pos2_, link_size='vary',         **sz))
rt.table(_tiles_, spacer=10, per_row=4)

In [ ]:
import random
from math import sqrt
_lu_ = {'x0':[], 'y0':[], 'x1':[], 'y1':[]}
for i in range(100):
    #_lu_['x0'].append(16.0 + 496.0 * random.random())
    #_lu_['y0'].append(16.0 + 496.0 * random.random())
    _lu_['x0'].append(256.0)
    _lu_['y0'].append(256.0)
    _lu_['x1'].append(16.0 + 496.0 * random.random())
    _lu_['y1'].append(16.0 + 496.0 * random.random())
df = pl.DataFrame(_lu_)
df = df.with_columns(dx  = pl.col('x1')-pl.col('x0'),
                     dy  = pl.col('y1')-pl.col('y0'),
                     xm  = (pl.col('x0') + pl.col('x1'))/2.0,
                     ym  = (pl.col('y0') + pl.col('y1'))/2.0).\
        with_columns(mag = (pl.col('dx')**2 + pl.col('dy')**2).sqrt()).\
        with_columns(u   = pl.when(pl.col('mag') == 0).then(0).otherwise(pl.col('dx')/pl.col('mag')),
                     v   = pl.when(pl.col('mag') == 0).then(0).otherwise(pl.col('dy')/pl.col('mag'))).\
        with_columns(pu  = -1.0 * pl.col('v'),
                     pv  =        pl.col('u')).\
        with_columns(xo0 = pl.col('x0') + pl.col('mag')*pl.col('u')/3.0 + pl.col('mag')*pl.col('pu')/5.0,
                     yo0 = pl.col('y0') + pl.col('mag')*pl.col('v')/3.0 + pl.col('mag')*pl.col('pv')/5.0,
                     xo1 = pl.col('x1') - pl.col('mag')*pl.col('u')/3.0 + pl.col('mag')*pl.col('pu')/5.0,
                     yo1 = pl.col('y1') - pl.col('mag')*pl.col('v')/3.0 + pl.col('mag')*pl.col('pv')/5.0)
ops = [pl.lit('<path d="M '), pl.col('x0'), pl.lit(' '), pl.col('y0'), pl.lit(' C '),
       pl.col('xo0'), pl.lit(' '), pl.col('yo0'), pl.lit(' '),
       pl.col('xo1'), pl.lit(' '), pl.col('yo1'), pl.lit(' '),
       pl.col('x1'), pl.lit(' '), pl.col('y1'), pl.lit('" fill="none" stroke="black" stroke-width="1" />')]
ops = pl.concat_str(ops).alias('svg')
df  = df.with_columns(ops)
svg = ['<svg x="0" y="0" width="512" height="512" viewBox="0 0 512 512" xmlns="http://www.w3.org/2000/svg"> <rect x="0" y="0" width="512" height="512" fill="white" />']
svg.extend(df['svg'].to_list())
svg.append('</svg>')
rt.tile([''.join(svg)])